<a href="https://colab.research.google.com/github/ahmedHanzala/urdu-voice-cloning/blob/main/notebooks/dataset_preperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>
Dataset Preperation
</h1>
<p>This is the code used for preparing the datasets. There is alot of hard-coded paths added in the code, if you want to use this code you might have to do some editing. ill make this more general if i get time ;) </p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!pip install pydub
!pip install yt-dlp


<h2>
Downloading videos from youtube
</h2>

In [ ]:
#download video from youtube
!yt-dlp https://www.youtube.com/watch?v=L8LndKrrsOE&ab_channel=AskGanjiswag


!yt-dlp https://www.youtube.com/watch?v=jXO1-OWBWHo&ab_channel=AskGanjiswag

!yt-dlp https://www.youtube.com/watch?v=d5Ce3xFX0NQ&ab_channel=AskGanjiswag
!yt-dlp https://www.youtube.com/watch?v=rRFA2OF9ZO4&ab_channel=AskGanjiswag
!yt-dlp https://www.youtube.com/watch?v=3vOMenk3Yp4&ab_channel=AskGanjiswag

!yt-dlp https://www.youtube.com/watch?v=l8Y5qwGyLy8&ab_channel=AskGanjiswag

!yt-dlp https://www.youtube.com/watch?v=Ly4Tvwl8OQA

!yt-dlp https://www.youtube.com/watch?v=Dw-ZrwcPrH4

!yt-dlp https://www.youtube.com/watch?v=9NK5i6u1rxo&ab_channel=AskGanjiswag

!yt-dlp https://www.youtube.com/watch?v=PYw4py-ykso&ab_channel=AskGanjiswag
!yt-dlp https://www.youtube.com/watch?v=FFZxsVaoE7w&ab_channel=AskGanjiswag

!yt-dlp https://www.youtube.com/watch?v=K4FKdoQXSsM
!yt-dlp https://www.youtube.com/watch?v=Dw-ZrwcPrH4
!yt-dlp https://www.youtube.com/watch?v=0P3WBPwbKbU
!yt-dlp https://www.youtube.com/watch?v=oByiu1TVWWI
!yt-dlp https://www.youtube.com/watch?v=mclKOoczZm0
!yt-dlp https://www.youtube.com/watch?v=DXYuCrT4YsA
!yt-dlp https://www.youtube.com/watch?v=JQo3-g1hImQ&ab_channel=AskGanjiswag
!yt-dlp https://www.youtube.com/watch?v=FDOm7GLnNmw&ab_channel=AskGanjiswag



<h2>Renaming the videos as 1,2,..,n</h2>

In [ ]:
import os

directory = "/content"
extension = ".webm"

# Get the list of files in the directory
file_list = os.listdir(directory)
print(file_list)

# Sort the files in alphanumeric order
file_list.sort()

# Rename the files from 1 to 19
for i, filename in enumerate(file_list[:19], start=1):
    if filename.endswith(extension):
        new_filename = str(i) + extension
        current_path = os.path.join(directory, filename)
        new_path = os.path.join(directory, new_filename)
        os.rename(current_path, new_path)
        print(f"Renamed {filename} to {new_filename}")

<h2>Converting into wav files</h2>

In [ ]:
#change to wav format
from pydub import AudioSegment
import subprocess

for i in range(18):
  # Load the WebM video file
  video_file = f"{i+1}.webm"

  # Extract the audio from the video and save it as a temporary file
  temp_file = "temp_audio.wav"
  subprocess.call(['ffmpeg', '-i', video_file, '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', temp_file])

  # Load the audio from the temporary file
  audio = AudioSegment.from_file(temp_file, format="wav")

  # Export the audio as a new WAV file
  audio.export(f"output_file{i+1}.wav", format="wav")

  # Delete the temporary file
  subprocess.call(['rm', temp_file])

In [ ]:
import IPython
IPython.display.Audio(filename='output_file.wav')

<h2>Converting from 44k to 22k Hz</h2>

In [ ]:
#convert to 22kHz sampling rate
from pydub import AudioSegment

for i in range(18):
  # Load the audio file
  audio = AudioSegment.from_wav(f"output_file{i+1}.wav")

  # Set the new sample rate
  new_sample_rate = 22050

  # Change the sample rate
  audio = audio.set_frame_rate(new_sample_rate)

  # Export the audio file with the new sample rate
  audio.export(f"output_file{i+1}.wav", format="wav")

<h2>Splitting the wav into chunks on silence</h2>

In [ ]:
#splitting into chunks

from pydub import AudioSegment
from pydub.silence import split_on_silence
audio_chunks = []
for i in range(18):
  #reading from audio mp3 file
  path=f"output_file{i+1}.wav"
  sound = AudioSegment.from_wav(path)
  # spliting audio files
  print(f"splitting {path}")
  print(len(audio_chunks))
  x = split_on_silence(sound, min_silence_len=500, silence_thresh=-40 )
  audio_chunks.append(x)

In [ ]:
output_path="/content/drive/MyDrive/junaid-akram-dataset/wavs/"
for j in range(len(audio_chunks)):
  for i, chunk in enumerate(audio_chunks[j]):
      # Check if the chunk has non-zero duration
      if len(chunk) > 0:
          # Set the output file path
          orignal = f"{output_path}{j}-{i}.wav"
          silenced_segment = AudioSegment.silent(duration=500)
          chunk_with_silence = chunk + silenced_segment
          # Export the audio chunk as a WAV file
          chunk_with_silence.export(orignal, format="wav")

In [ ]:
import os

path = "/content/drive/MyDrive/junaid-akram-dataset/wavs/"
num_files = len(os.listdir(path))

print(f"There are {num_files} files in the {path} directory.")

There are 14078 files in the /content/drive/MyDrive/junaid-akram-dataset/wavs/ directory.


<h1> Using Whisper AI for transcription</h1>

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = whisper.load_model("large")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|██████████████████████████████████████| 2.87G/2.87G [00:30<00:00, 102MiB/s]


Model is multilingual and has 1,541,384,960 parameters.


<h2>Transcribing into Urdu</h2>

In [ ]:
directory_path="/content/drive/MyDrive/junaid-akram-dataset/wavs/"
string=""
file_path = "/content/drive/MyDrive/junaid-akram-dataset/transcription.txt"
count =0
# Open the file in write mode and write some text to it

for file_name in os.listdir(directory_path):
  present  = False
  if file_name != ".ipynb_checkpoints":
    with open(file_path, "r") as f:
      # Read the contents of the file into a string
      contents = f.read()
      # Check if the phrase is in the contents
      if f"wavs/{file_name}|" in contents:
        present=True
    if present == False:
      audio = whisper.load_audio(f"{directory_path}{file_name}")
      print(f"File transcribing: {file_name}")
      audio = whisper.pad_or_trim(audio)
      mel = whisper.log_mel_spectrogram(audio).to(model.device)
      # detect the spoken language
      options = whisper.DecodingOptions(language="ur")
      result = whisper.decode(model, mel, options)
      string = f"wavs/{file_name}|{result.text}\n"
      count = count+1
      print(count)
      with open(file_path, "a") as f:
        f.write(string)



In [ ]:
directory_path="/content/drive/MyDrive/urdu-imran-dataset/wavs/"
string=""
file_path = "/content/drive/MyDrive/dataset/transcription.txt"
# Open the file in write mode and write some text to it
for file_name in os.listdir(directory_path):
  result = model.transcribe(f"{directory_path}{file_name}")
  string = string+ f"wavs/{file_name}|{result['text']}\n"

with open(file_path, "w") as f:
  f.write(string)


In [ ]:
import random

# Set the file paths
transcription_file = "/content/drive/MyDrive/urdu-imran-dataset/transcription.txt"
train_file = "/content/drive/MyDrive/urdu-imran-dataset/train.txt"
val_file = "/content/drive/MyDrive/urdu-imran-dataset/val.txt"

# Set the train/val split ratio
train_ratio = 0.8

# Read the lines from the transcription file
with open(transcription_file, "r") as f:
    lines = f.readlines()

# Shuffle the lines randomly
random.shuffle(lines)

# Compute the number of lines for the train and val sets
num_train = int(train_ratio * len(lines))
num_val = len(lines) - num_train

# Write the lines to the train file
with open(train_file, "w") as f:
    f.writelines(lines[:num_train])

# Write the lines to the val file
with open(val_file, "w") as f:
    f.writelines(lines[num_train:])

# Print some statistics
print(f"Number of lines in train file: {num_train}")
print(f"Number of lines in val file: {num_val}")

Number of lines in train file: 2061
Number of lines in val file: 516
